In [1]:
import pandas as pd
import glob

pd.set_option("display.precision", 2) #always just show 2 decimal points


#Read in all country data
path = r'/Users/rvpazos/Documents/mvtec/ManifestoProject/workshop2-manifesto-rebeccaFolder/country-data' # use your path
all_files = glob.glob(path + "/*.csv")

all_files
li = []
for filename in all_files:
    df = pd.read_csv(filename,header=0)
    li.append(df)

allData = pd.concat(li, axis=0)

# allData.reset_index()
# allData = allData.set_index(pd.DatetimeIndex(allData['index'])).drop(['index'], axis=1)

allData = allData.rename(columns={'index':'yearObject'})

allData



,yearObject,parfam,countryname,environ,culture,equality,pervote,rile,electionDate,partyname
0,1989-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
1,1990-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
2,1991-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
3,1992-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
4,1993-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left
...,...,...,...,...,...,...,...,...,...,...
228,2016-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'
229,2017-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'
230,2018-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'
231,2019-01-01,95,Estonia,0.0,0.0,0.0,7.12,62.5,1992-09-01,Electoral Union ‘Independent Royalists'


In [2]:
#Parfam renamed
import numpy as np 

#Change parfam codes to abbreviations for better identification
conditions = [
    (allData['parfam'] == 10),
    (allData['parfam'] == 20),
    (allData['parfam'] == 30),
    (allData['parfam'] == 40),
    (allData['parfam'] == 50),
    (allData['parfam'] == 60),
    (allData['parfam'] == 70),
    (allData['parfam'] == 80),
    (allData['parfam'] == 90),
    (allData['parfam'] == 95),
    (allData['parfam'] == 98),
]

values = ['ECO','LEF','SOC','LIB','CHR', 'CON', 'NAT', 'AGR', 'ETH', 'SIP', 'DIV']

allData['parfamName'] = np.select(conditions,values)


allData.head()
# allData.to_csv('allCountries.csv')



,yearObject,parfam,countryname,environ,culture,equality,pervote,rile,electionDate,partyname,parfamName
0,1989-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
1,1990-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
2,1991-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
3,1992-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO
4,1993-01-01,10,Netherlands,8.7,3.4,6.1,4.07,-15.0,1989-09-01,Green Left,ECO


In [51]:
#reset all data for JSON output

df2 = allData.set_index('parfamName')
df2
df2['yearObject'] = pd.to_datetime(df['yearObject'])

KeyError: 'yearObject'

In [31]:
import numpy as np

# allData['parfam'] = df['parfam'].astype('category')

pt = pd.pivot_table(allData, index='parfamName', values=['index','parfam','environ','culture','equality','pervote','rile'], aggfunc={'index': 'min', 'parfam' : 'count', 'environ' : 'sum','culture' : 'sum','equality' : 'sum', 'pervote' : 'sum', 'rile' : 'mean'}, dropna=True)
pt = pt.rename(columns={'parfam':'parfamCount'})
# pt['avgVote'] = (pt['pervote'])/(pt['parfamCount'])

pt.head()

,culture,environ,equality,index,parfamCount,pervote,rile
parfamName,,,,,,,
AGR,856.38,2399.46,1906.87,1944-01-01,493,4126.22,5.41
CHR,2803.19,3962.39,4114.16,1945-01-01,1174,22878.17,4.17
CON,2127.70,2156.50,2777.19,1944-01-01,922,18229.83,7.49
DIV,171.72,194.93,72.43,1997-01-01,77,954.89,-4.67
ECO,1712.81,12363.51,3159.56,1981-01-01,593,2523.15,-12.10


In [17]:
import json

result = pt.to_json('test.json', orient='index')


pt['year'] = pt.index.year()



AttributeError: 'MultiIndex' object has no attribute 'year'

In [8]:
spainDF = allData.loc[(allData['countryname']=='Spain')] 

decadesFilter = ['1990-01-01','1995-01-01','2000-01-01','2005-01-01','2010-01-01','2015-01-01','2020-01-01']
filteredSpain = spainDF[spainDF.index.isin(decadesFilter)]
filteredSpain


ptSpain = pd.pivot_table(filteredSpain, index=['index','parfamName'], values=['parfam','per503','pervote','environWeighted'], aggfunc={'parfam' : 'count', 'per503' : 'sum', 'pervote' : 'sum', 'environWeighted' : 'sum'}, dropna=True)
ptSpain['avgVote'] = (ptSpain['pervote'])/(ptSpain['parfam'])

ptSpain.to_csv('spain-grouped.csv')

In [95]:
n_by_fam = allData.groupby(['index','parfam'])['parfam'].count()
n_by_fam


index       parfam
1944-01-01  20         1
            30         1
            40         1
            60         1
            80         1
                      ..
2020-01-01  70        24
            80        12
            90        13
            95        16
            98         4
Name: parfam, Length: 750, dtype: int64